In [17]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
import time
from webdriver_manager.chrome import ChromeDriverManager
import sqlite3

In [21]:
conn = sqlite3.connect('./parking.db')
cursor = conn.cursor()

df = pd.read_csv("식당정보_naver5.csv", encoding='cp949')
filtered_df = df[df['MCT_NM'].notna()]
mct_nm_list = filtered_df[['MCT_NM', 'placeID']].values.tolist()


driver = webdriver.Chrome()
driver.get("https://map.naver.com/")

print(mct_nm_list)

for restaurant, restaurant_id in mct_nm_list:
    partking_list = []

    if ' ' not in restaurant:
        restaurant += " 제주"
    print("========================================================")
    print("info \t |", restaurant, restaurant_id)

    try:
        search_box = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "input.input_search"))
        )
        search_box.clear()
        search_box.send_keys(restaurant)
        search_box.send_keys(Keys.RETURN)
    except Exception as e:
        print(f"error \t | input 못찾았거나.. send_keys에 오류가 났거나 했겠죠?", e)
        continue

    print("good \t | searching...... waiting 5s")
    time.sleep(5)

    try:
        clear_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable(
                (By.CSS_SELECTOR, "#home_search_input_box > div > div > button.btn_clear"))
        )
        clear_button.click()
    except Exception as e:
        print("error \t | 검색했던 단어를 지우지 못했어요.. x를 못찾거나 그런 거겠죠?", e)
        continue

    print("good \t | 검색했던 단어 지웠어요")

    try:
        search_box = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "input.input_search"))
        )
        search_box.send_keys("주차장")
        search_box.send_keys(Keys.RETURN)
    except Exception as e:
        print("error \t | '주차장' 이거 input에 넣지 못했어요...", e)
        continue

    print("good \t | 주차장 검색했어요... waiting 5s")
    time.sleep(5)

    try:
        iframe = WebDriverWait(driver, 2).until(
            EC.presence_of_element_located((By.ID, 'searchIframe'))
        )
        driver.switch_to.frame(iframe)

        parking_names = WebDriverWait(driver, 2).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, 't3s7S'))
        )[:5]

        parking_details = WebDriverWait(driver, 2).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, 'wNotu'))
        )[:5]

        for name, detail in zip(parking_names, parking_details):

            cursor.execute("INSERT INTO parking (placeID, parking_name, parking_detail) VALUES (?, ?, ?)",
                           (restaurant_id, name.text, detail.text))
            conn.commit()
            print(
                f"good info\t | db에 {restaurant_id} - {name.text} - {detail.text} 넣었어요")

    except Exception as e:
        print("error \t | iframe 들어오는데 문제가 발생했나오바ㅛㅇ...", e)

    try:
        driver.switch_to.default_content()
        clear_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable(
                (By.CSS_SELECTOR, "#home_search_input_box > div > div > button.btn_clear"))
        )
        clear_button.click()
    except Exception as e:
        print("error \t | 주차장 검색을 지우지 못했어요... ", e)
        continue

    print("good \t | 주차장 검색을 지웠어요.")
    time.sleep(1)

conn.close()
print("done!")

[['아웃백스테이크하우스 제주아일랜드점', 1418972938.0], ['베이힐', 1463424775.0], ['신세계푸드 제주신화월드  윤경양식당', 1199772319.0], ['신세계푸드 제주신화월드  티플레이스', 1471178944.0], ['신세계푸드 제주신화월드  화니제주신', 1420348866.0], ['아워홈 제주지방합동청사점', 1415013033.0], ['그랜드부민', 13581342.0], ['바다바라', 1529840514.0], ['산굼부리', 1789181978.0], ['삼다도횟집', 12883219.0], ['아워홈 제주국제공항점', 1332337704.0], ['아워홈 제주국제공항점', 1332337704.0], ['오드랑베이커리', 1729610778.0], ['오설록 제주점', 32520687.0], ['제주서서갈비', 1140636590.0], ['참숯구이몬트락', 37622421.0], ['커피빈코리아', 1695475563.0], ['파리크라상파리바게뜨 제주공항점', 31499631.0], ['파리크라상파리바게뜨제주공항탑승점', 1921928717.0], ['피자알볼로 제주점', 33211465.0], ['064제주인생곱창', 1220007047.0], ['1950에어차이나 제주공항점', 1424589909.0], ['24시국수회관', 16723803.0], ['24시누름돌김치찌개', 1028483996.0], ['24시누름돌생고기김치찌개 용담점', 1212258834.0], ['24시뼈다귀신제주점', 11730904.0], ['24시뼈다귀탕 신서귀점', 1869656386.0], ['24시뼈다귀탕노형점', 1264153834.0], ['24시뼈다귀탕동홍점', 1274041067.0], ['24시뼈다귀탕중문점', 1724563592.0], ['24시얼큰시원생고기김치찌개 노형점', 1736374666.0], ['24시얼큰시원생고기김치찌개아라점', 1925347467.0], ['24시얼큰시원생고기김치찌개외도점', 19

KeyboardInterrupt: 